## 1. 모델 구현
- https://www.tensorflow.org/guide/keras/sequential_model

```
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)
```

In [13]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

In [20]:
# padding='same' : 텐서플로우가 자동으로 패딩을 삽입해 입력값과 출력값의 크기를 맞춰줌
# padding='valid' : 텐서플로우가 자동으로 패딩을 적용하지 않고 필터를 적용해서 출력값의 크기가 작이짐

def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            layers.Input(input_shape),
            
            # 1st
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),    # 64 : 필터 수(출력), 3 : 3x3 필터 크기 => 각 필터의 가중치는 다름
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),    # 배치별 특징 값을 정규화(relu로 인한 다양한 값 생성(local optimal)) => 학습 속도 증가
            layers.Dropout(0.5),
            
            # 2nd
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),    # 128 : 필터 수(출력), 3 : 3x3 필터 크기 => 각 필터의 가중치는 다름
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),    # 배치별 특징 값을 정규화(relu로 인한 다양한 값 생성(local optimal)) => 학습 속도 증가
            layers.Dropout(0.3),
            
            # FC
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),    # 2nd에서 출력이 128개
            layers.Dense(1, activation='sigmoid')    # 마지막 결과 : 맞는지 여부(2진 논리)
        ]
    )
    
    return model

In [21]:
input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

# 파라미터(param)가 크면 클수록 연산 수가 많아져 더 정확해짐

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_17 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_18 (Conv2D)          (None, 128, 128, 128)     7

In [22]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),    # 64 : 필터 수(출력), 3 : 3x3 필터 크기 => 각 필터의 가중치는 다름
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),    # 배치별 특징 값을 정규화(relu로 인한 다양한 값 생성(local optimal)) => 학습 속도 증가
                layers.Dropout(0.5)
            ], name='conv_block_1'
        )

        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),    # 64 : 필터 수(출력), 3 : 3x3 필터 크기 => 각 필터의 가중치는 다름
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),    # 배치별 특징 값을 정규화(relu로 인한 다양한 값 생성(local optimal)) => 학습 속도 증가
                layers.Dropout(0.5)
            ], name='conv_block_2'
        )

        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation='relu'),    # 2nd에서 출력이 128개
                layers.Dense(1, activation='sigmoid')    # 마지막 결과 : 맞는지 여부(2진 논리)        
        ], name='classifier'
        )
        
    def call(self, input_tensor, training=False):
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [24]:
input_shape = (None, 256, 256, 3)    # None : 배치 사이즈 설정 안함
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block_1 (Sequential)   (None, 128, 128, 64)      38976     
                                                                 
 conv_block_2 (Sequential)   (None, 64, 64, 128)       221952    
                                                                 
 classifier (Sequential)     (None, 1)                 16641     
                                                                 
Total params: 277,569
Trainable params: 277,185
Non-trainable params: 384
_________________________________________________________________


In [25]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)